In [89]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun(backend="news")
search.run("Avoid these trigger foods if you want to quit smoking in 2024")

'Alcohol Trigger Explanation: Alcohol and smoking often accompany each other. The relaxing effect of alcohol can weaken inhibitions, making it harder to resist the urge to smoke. Advertisement Alternatives: Choose non-alcoholic beverages or limit alcohol consumption during the initial stages of quitting. Sugary Snacks Health Care Breaking News Health Want to quit smoking? These trigger foods are known to cause nicotine cravings — avoid them like the plague By David Landsel Published Jan. 1,... What you eat while going cold turkey can make a big difference — and some foods could derail even the most committed quitter, an expert from gosmokefree.co.uk told The Mirror. There are four types of foods — and one very popular drink — that wannabe ex-smokers should write off completely as they struggle with the popular new year\'s ... Spicy foods, sugary snacks, fried foods (sorry french fry lovers), and foods with high sodium content are all considered no-gos. "Spicy and sweet foods can trigge

In [159]:
# 現在の日本時間と1日前の日付を取得
now_jst = datetime.utcnow() + timedelta(hours=9)  # JSTはUTCより9時間進んでいる
from_date = (now_jst - timedelta(days=7)).strftime('%Y-%m-%d')
to_date = now_jst.strftime('%Y-%m-%d')

# ニュースのクエリと取得
query = 'technology'
articles = newsapi.get_everything(q=query, from_param=from_date, to=to_date, sort_by='relevancy', page=5)
print(articles)
# 現在のニュースタイトルセット
current_titles = set(article['title'] for article in articles['articles'])

# 新しいニュースがあるか確認
new_titles = current_titles - last_titles
if new_titles:
    # 新しいニュースがあれば処理を実行
    search_query = next(iter(new_titles))  # 最初の新しいタイトルを取得
    print(search_query)
    search = DuckDuckGoSearchRun(backend="news")
    detail_search_result = search.run(search_query)
    # 現在のタイトルセットを保存
    last_titles = current_titles
    information = str(articles['articles'][0])

{'status': 'ok', 'totalResults': 11718, 'articles': [{'source': {'id': None, 'name': 'Ghacks Technology News'}, 'author': 'Ashwin', 'title': 'Microsoft removes WordPad from clean installs of Windows 11 Canary Builds, and you cannot reinstall it', 'description': 'Microsoft has begun the removal of WordPad from Windows 11 Insider Program. The app is no longer available on clean installs of the Canary Channel Builds. This is not surprising, as the […]\nThank you for being a Ghacks reader. The post Microsoft removes WordP…', 'url': 'https://www.ghacks.net/2024/01/05/microsoft-removes-wordpad-from-clean-installs-of-windows-11-canary-builds-and-you-cannot-reinstall-it/', 'urlToImage': 'https://www.ghacks.net/wp-content/uploads/2024/01/Microsoft-removes-WordPad-from-clean-installs-of-Windows-11-Canary-Builds-and-you-cannot-reinstall-it.jpg', 'publishedAt': '2024-01-05T10:47:13Z', 'content': 'Microsoft has begun the removal of WordPad from Windows 11 Insider Program. The app is no longer avail

## 自動投稿パイプライン

In [ ]:
from newsapi import NewsApiClient
import pandas as pd
from langchain.tools import DuckDuckGoSearchRun
import openai
import tweepy
from datetime import datetime, timedelta
import openai
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

#環境変数読み込み
load_dotenv()

# News APIの設定
newsapi = NewsApiClient(api_key=os.getenv("NEWS_API_KEY"))

# OpenAI APIの設定
openai.api_key = os.getenv("OPENAI_KEY")

#Twitter APIの設定
consumer_key = os.getenv("TWITTER_KEY")
consumer_secret = os.getenv("TWITTER_SECRET")

def make_post_txt(information, detail_search_result) -> str:
    # タスク
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": 'あなたは博識で高い煽り能力を持った２chネラーです。'},
            {"role": "user", "content": f"""
            News APIで帰ってきたニュース情報を与えます。
            併せて、ニュースに関する検索結果を情報として与えます。
            この情報を基にしてTwitterにコメント付きでポストする際の適切なテキストを作成して下さい。
            その際、以下の点に留意して下さい。
            ・生成するテキストは絶対にリンク込みで日本語200文字相当に収まるようにして下さい
            ・あなたは可愛いアイドルです。
            """},
            {"role": "assistant", "content": f"""
            了解だよ！
            """},
            {"role": "user", "content": f"""
            'title': 'Avoid these trigger foods if you want to quit smoking in 2024',
            'description': 'Ekta Singhwal, dietician Ujala Cygnus Group of Hospitals, explained that certain foods contain trace amounts of nicotine. "While the levels are much lower than in tobacco, consuming them can potentially trigger a mild nicotine response," she said.',
            'url': 'https://indianexpress.com/article/lifestyle/life-style/trigger-foods-quit-smoking-2024-9096781/',
            'urlToImage': 'https://images.indianexpress.com/2024/01/smoking-cigarettes.jpg',
            'publishedAt': '2024-01-05T12:50:29Z',
            'web_info': 'Alcohol Trigger Explanation: Alcohol and smoking often accompany each other. The relaxing effect of alcohol can weaken inhibitions, making it harder to resist the urge to smoke. Advertisement Alternatives: Choose non-alcoholic beverages or limit alcohol consumption during the initial stages of quitting. Sugary Snacks Health Care Breaking News Health Want to quit smoking? These trigger foods are known to cause nicotine cravings — avoid them like the plague By David Landsel Published Jan. 1,... What you eat while going cold turkey can make a big difference — and some foods could derail even the most committed quitter, an expert from gosmokefree.co.uk told The Mirror. There are four types of foods — and one very popular drink — that wannabe ex-smokers should write off completely as they struggle with the popular new year\'s ... Spicy foods, sugary snacks, fried foods (sorry french fry lovers), and foods with high sodium content are all considered no-gos. "Spicy and sweet foods can trigger nicotine, so it\'s important to be mindful of your diet." [cravings]\'\' said an anonymous expert. We can help. Sarasota Memorial hosts monthly classes on smoking cessation, led by the experts of Tobacco Free Florida. In these FREE sessions, develop a personalized quit plan and double your chances of success. Check the SMH Calendar for the date of the next session. To register for an upcoming class near you, call (866) 534-7909 or click here.'
            """},
            {"role": "assistant", "content": f"""
            みんな、禁煙のコツをシェアするね！🌟🚭 お酒や甘いもの、スパイシーな食べ物がタバコの欲求を引き起こすって知ってた？🤔 健康のためにも、これらを控えようね✨ 一緒に頑張ろう！💪💕 #禁煙 #健康生活 #ニコチン対策 詳細はこちら😊👉: https://indianexpress.com/article/lifestyle/life-style/trigger-foods-quit-smoking-2024-9096781/
            """},
            {"role": "user", "content":information+f"web_info, {detail_search_result}"}
        ]
    )
    post_txt = response.choices[0].message.content
    return post_txt

def post_tweet(post_txt):

    # ツイート内容
    payload = {"text": post_txt}
    
    # アクセストークンの保存ファイル
    token_file = 'twitter_tokens.json'
    
    # アクセストークンを読み込む
    if os.path.exists(token_file):
        with open(token_file, 'r') as file:
            tokens = json.load(file)
            access_token = tokens.get('access_token')
            access_token_secret = tokens.get('access_token_secret')
    else:
        access_token = None
        access_token_secret = None
    
    # アクセストークンが存在しない場合のみ新たに認証フローを実行
    if access_token and access_token_secret:
        oauth = OAuth1Session(
            consumer_key,
            client_secret=consumer_secret,
            resource_owner_key=access_token,
            resource_owner_secret=access_token_secret,
        )
    else:
        # Get request token
        request_token_url = "https://api.twitter.com/oauth/request_token?oauth_callback=oob&x_auth_access_type=write"
        oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)
    
        try:
            fetch_response = oauth.fetch_request_token(request_token_url)
        except ValueError:
            print("There may have been an issue with the consumer_key or consumer_secret you entered.")
    
        resource_owner_key = fetch_response.get("oauth_token")
        resource_owner_secret = fetch_response.get("oauth_token_secret")
    
        # Get authorization
        base_authorization_url = "https://api.twitter.com/oauth/authorize"
        authorization_url = oauth.authorization_url(base_authorization_url)
        print("Please go here and authorize: %s" % authorization_url)
        verifier = input("Paste the PIN here: ")
    
        # Get the access token
        access_token_url = "https://api.twitter.com/oauth/access_token"
        oauth = OAuth1Session(
            consumer_key,
            client_secret=consumer_secret,
            resource_owner_key=resource_owner_key,
            resource_owner_secret=resource_owner_secret,
            verifier=verifier,
        )
        oauth_tokens = oauth.fetch_access_token(access_token_url)
    
        access_token = oauth_tokens["oauth_token"]
        access_token_secret = oauth_tokens["oauth_token_secret"]
    
        # トークンを保存
        tokens = {
            'access_token': access_token,
            'access_token_secret': access_token_secret
        }
        with open(token_file, 'w') as file:
            json.dump(tokens, file)
        
        # OAuthセッションの設定
        oauth = OAuth1Session(
            consumer_key,
            client_secret=consumer_secret,
            resource_owner_key=access_token,
            resource_owner_secret=access_token_secret,
        )
    
    # Making the request
    response = oauth.post(
        "https://api.twitter.com/2/tweets",
        json=payload,
    )
    
    # エラーチェックとレスポンスの表示
    if response.status_code != 201:
        raise Exception(
            "Request returned an error: {} {}".format(response.status_code, response.text)
        )
    
    print("Response code: {}".format(response.status_code))
    json_response = response.json()
    return json.dumps(json_response, indent=4, sort_keys=True)

def get_news(query, duration, sort_method, pages):

    # 現在の日本時間と1日前の日付を取得
    now_jst = datetime.utcnow() + timedelta(hours=9)  # JSTはUTCより9時間進んでいる
    from_date = (now_jst - timedelta(days=duration)).strftime('%Y-%m-%d')
    to_date = now_jst.strftime('%Y-%m-%d')

    # 処理済みタイトルのファイルパス
    processed_titles_file = 'processed_titles.json'
    
    # ニュースのクエリと取得
    query = 'technology'
    # query = 'finance'
    articles = newsapi.get_everything(q=query, from_param=from_date, to=to_date, sort_by=sort_method, page=pages)

    return articles

    
# 前回取得したニュースタイトルの保存用
last_titles = set()

while True:

    #ニュース記事を確認
    articles = get_news('technology', 3, 'relevancy', 2)
    # 処理済みタイトルの読み込み
    if os.path.exists(processed_titles_file):
        with open(processed_titles_file, 'r') as file:
            processed_titles = set(json.load(file))
    else:
        processed_titles = set()

    # 各ニュースに対して順に処理
    for article in articles['articles']:
        title = article['title']
        if title not in processed_titles:
            print('処理中のニュースタイトル:', title)

            # 新しいニュースがあれば処理を実行
            print('新規サーチクエリ:', title)
            search = DuckDuckGoSearchRun(backend="news")
            detail_search_result = search.run(title)
            print('検索結果:', detail_search_result)
    
            # タイトルに'home'が含まれる記事のリスト
            information = str([article for article in articles['articles'] if title.lower() in article['title'].lower()])
            print('インフォメーション:', information)
            
            #Twitter用のポストを作成
            post_txt = make_post_txt(information, detail_search_result)
            print('ツイート:', post_txt)
            print('投稿時間', now_jst)
    
            #Twitterに投稿
            try:
                post_tweet(post_txt)
            except Exception as e:
                print(f"ツイートの投稿中にエラーが発生しました: {e}")
                continue  # エラーが発生したら、次のニュースへ進む
    
            # 処理されたニュースタイトルを保存
            processed_titles.add(title)
            with open(processed_titles_file, 'w') as file:
                json.dump(list(processed_titles), file)
        
            # ３０分待機
            time.sleep(30 * 60)
        else:
            print('No new titles was found.' + str(now_jst))
            time.sleep(30*60)

処理中のニュースタイトル: SEC denies Apple's bid to stop shareholders questioning its use of AI
新規サーチクエリ: SEC denies Apple's bid to stop shareholders questioning its use of AI
検索結果: William Gallagher | Jan 04, 2024 Apple Park 0 Facebook x.com Reddit The Securities and Exchange Commission has rejected requests by both Apple and Disney to exclude AI from discussions at... U.S. SEC: Apple cannot stop avoid shareholder votes regarding AI usage Thursday, January 4, 2024 2:36 pm No Comments News Apple and Disney cannot avoid shareholder votes about their... Reuters reports that Apple and Disney can't avoid shareholder votes about their use of artificial intelligence (AI) put forward by a labor group, the top U.S. securities regulator has ruled.. In notices dated Jan. 3, the U.S. Securities and Exchange Commission rejected requests by the two companies to exclude from their upcoming annual meetings calls for reports on their use of AI. Jan 5, 2024 SEC rules Apple and Disney can't dodge AI disclosure vote

In [114]:
from requests_oauthlib import OAuth1Session
import os
import json

# 環境変数からAPIキーとシークレットを取得
consumer_key = '2n3d06cqz63v1miP5pEyJ7H6g'
consumer_secret = '5XXPMQSxsY6BC46BThbmO0WKEPIMaWVDYH6u6LKOtkMPMyHucD'

# ツイート内容
payload = {"text": "Hello world!!!!"}

# アクセストークンの保存ファイル
token_file = 'twitter_tokens.json'

# アクセストークンを読み込む
if os.path.exists(token_file):
    with open(token_file, 'r') as file:
        tokens = json.load(file)
        access_token = tokens.get('access_token')
        access_token_secret = tokens.get('access_token_secret')
else:
    access_token = None
    access_token_secret = None

# アクセストークンが存在しない場合のみ新たに認証フローを実行
if access_token and access_token_secret:
    oauth = OAuth1Session(
        consumer_key,
        client_secret=consumer_secret,
        resource_owner_key=access_token,
        resource_owner_secret=access_token_secret,
    )
else:
    # Get request token
    request_token_url = "https://api.twitter.com/oauth/request_token?oauth_callback=oob&x_auth_access_type=write"
    oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

    try:
        fetch_response = oauth.fetch_request_token(request_token_url)
    except ValueError:
        print("There may have been an issue with the consumer_key or consumer_secret you entered.")

    resource_owner_key = fetch_response.get("oauth_token")
    resource_owner_secret = fetch_response.get("oauth_token_secret")

    # Get authorization
    base_authorization_url = "https://api.twitter.com/oauth/authorize"
    authorization_url = oauth.authorization_url(base_authorization_url)
    print("Please go here and authorize: %s" % authorization_url)
    verifier = input("Paste the PIN here: ")

    # Get the access token
    access_token_url = "https://api.twitter.com/oauth/access_token"
    oauth = OAuth1Session(
        consumer_key,
        client_secret=consumer_secret,
        resource_owner_key=resource_owner_key,
        resource_owner_secret=resource_owner_secret,
        verifier=verifier,
    )
    oauth_tokens = oauth.fetch_access_token(access_token_url)

    access_token = oauth_tokens["oauth_token"]
    access_token_secret = oauth_tokens["oauth_token_secret"]

    # トークンを保存
    tokens = {
        'access_token': access_token,
        'access_token_secret': access_token_secret
    }
    with open(token_file, 'w') as file:
        json.dump(tokens, file)
    
    # OAuthセッションの設定
    oauth = OAuth1Session(
        consumer_key,
        client_secret=consumer_secret,
        resource_owner_key=access_token,
        resource_owner_secret=access_token_secret,
    )

# Making the request
response = oauth.post(
    "https://api.twitter.com/2/tweets",
    json=payload,
)

# エラーチェックとレスポンスの表示
if response.status_code != 201:
    raise Exception(
        "Request returned an error: {} {}".format(response.status_code, response.text)
    )

print("Response code: {}".format(response.status_code))
json_response = response.json()
print(json.dumps(json_response, indent=4, sort_keys=True))


Response code: 201
{
    "data": {
        "edit_history_tweet_ids": [
            "1743664091858939960"
        ],
        "id": "1743664091858939960",
        "text": "Hello world!!!!"
    }
}
